# urls to download data set

In [22]:
url= "https://raw.githubusercontent.com/N-Chandru/MTP/main/match_data_2010_2011.csv"
url1="https://raw.githubusercontent.com/N-Chandru/MTP/main/match_data_1995.csv"

# Importing required packages

In [23]:
%%capture
!pip install fancyimpute

In [24]:
import pandas as pd
import numpy as np
import math
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler

# Data pre processing

In [25]:
# considering only playerA and layerB column in te dataframe
def Data(Urls):
  frames = []
  for url in Urls:
    df = pd.read_csv(url)
    df = df[["playerA", "playerB"]]
    frames.append(df)
    data = pd.concat(frames)
  
  return data[data.playerB != 'N/A Bye']

In [ ]:
data = Data([url, url1])
data

In [27]:
# creating players dictionary 
# players names as key and id as value....
def Playerss(dataframe):
  A = dataframe['playerA'].tolist()
  B = dataframe['playerB'].tolist()
  Players = list(set(A) | set(B))
  Players_dict = {val: id for id, val in enumerate(Players)}
  return Players_dict

# Pairwise Block Rank Algo.

## computing Pairwise Preference Matrix:-

${Pr(i≻j)}$ ≡ $\frac{1}{1+exp−(si−sj)}$

In [28]:
def Count(dataframe):
  count = {}
  Data = dataframe.values.tolist()

  for data in Data:
    data = tuple(data)
    if data in count:
      count[data] += 1
    else:
      count[data] = 1
  
  return count

def probability(Matrix):
  n, _ = Matrix.shape
  prob = np.zeros(shape = (n, n), dtype=float)
  skew = np.zeros(shape = (n, n), dtype=float)

  prob = 1.0/(1.0 + np.exp(Matrix.T- Matrix))
  
  with np.errstate(invalid='ignore', divide='ignore'):
    skew = np.log10(prob/prob.T)

  skew = np.nan_to_num(skew, nan=0, posinf=0, neginf=0)
  return skew

# return pairwise Preference matrix
def PPM(dataframe, Players):

  preferenceMatrix = np.zeros(shape = (len(Players), len(Players)), dtype=float)
  count = Count(dataframe)
  
  for p1 in Players:
    for p2 in Players:
      if p1!=p2:
        preferenceMatrix[Players[p1]][Players[p2]] = count.get((p1, p2), 0)
        preferenceMatrix[Players[p2]][Players[p1]] = count.get((p2, p1), 0)
  
  return probability(preferenceMatrix)

# returns a matrix of 0's and 1's if given input matrix has missing value means
# fill with 0 and known value means with 1
def mask(matrix):
  matrix[matrix!=0]=1
  return matrix

## Matrix completion
Reference [link ](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=6751420)

**Unifying Nuclear Norm and Bilinear Factorization
Approaches for Low-rank Matrix Decomposition**

Input: $X, W ∈ R_{M ×N}, μ, λ, ρ$

while *not converged* do:
>while not converged do:

> >Update $ U = (ρZ + Y) *V * (ρV^{T}  V + λI_r)^{-1}$

>>Update $V = (ρZ + Y)*U*(ρU^{T}U + λI_r)^{-1}$

>>Update$Z = W.( \frac{1}{2 + ρ}(2X + ρ(UV^{T} − ρ^{-1}Y)))+ W .(UV^{T} − ρ^{−1}Y)$

>end while
  
$Y = Y + ρ(Z − UV^{T})$

$ρ = min (ρμ, 1e^{20} )$

end while

Output: Complete Matrix  $ Z = UV^{T}$ 

In [29]:
# Matrix completion described in the above
def Matrix_Completion(Matrix, r, Mask, mu=0.01, ro=1, l=0.001, eps=1e-5, steps=1000):
  U = np.random.normal(size=(Matrix.shape[0], r))
  V = np.random.normal(size=(Matrix.shape[1], r))
  Z = U@V.T
  Y = np.zeros(shape=(Matrix.shape))
  I = np.eye(r)
  step = 0
  converged, converged2 = False, False

  while not converged:
    step2 =0
    while not converged2:
      U = (ro*Z +Y) @ V @ np.linalg.inv(ro*V.T @ V + l*I )
      V = (ro*Z +Y).T @ U @ np.linalg.inv(ro*U.T @ U + l*I )
      temp = U @V.T -1/ro * Y
      ZZ = Mask*(1/(2+ro) *(2*Matrix + ro*temp)) + (1-Mask)*temp
      converged2 = np.linalg.norm(Z-ZZ)/np.linalg.norm(Z) <eps or step2>steps
      step2+=1
      Z = ZZ.copy()
    temp = Z - U @ V.T
    Y = Y + ro*temp
    ro = min(ro*mu, 1e20)
    converged = np.linalg.norm(temp)<eps or step>steps
    step+=1
  
  Z = U @ V.T
  with np.errstate(invalid='ignore', divide='ignore'):
    Matrix = np.log10(abs(Z/Z.T))
  return Matrix


In [30]:
# fancyImpute matrix comletion module
# Takes Matrix with missing values as nan 
def fancyimpute(Matrix):
  
  Matrix[Matrix==0]=np.nan
  X_incomplete_normalized = BiScaler().fit_transform(Matrix)
  P_completed = SoftImpute().fit_transform(X_incomplete_normalized)
  np.fill_diagonal(P_completed, 0)
  with np.errstate(invalid='ignore', divide='ignore'):
    Matrix = np.log10(abs(P_completed/P_completed.T))

# Tournament Construction


In [31]:
class Tournaments():
  def __init__(self, Matrix):
    self.Tournament = Matrix.copy()
    # if T_ij is greater than zero means put an edge between i->j...
    self.Tournament[self.Tournament>0]=1
    self.Tournament[self.Tournament<0]=0

  # return a subtournament 
  def Sub_tournament(self, Vertices):
    
    n = len(Vertices)
    Sub_tournament = np.zeros(shape = (n, n), dtype = int)
    
    for i, u in enumerate(Vertices):
      for j, v in enumerate(Vertices):
        if u!=v and self.Tournament[u][v] == 1:
          Sub_tournament[i][j]=1
    return Sub_tournament

# Rank2Rank Algo and BR2R Algo..

In [32]:
import collections
def flatten(lis):
  for item in lis:
    if isinstance(item, collections.Iterable) and not isinstance(item, str):
      for x in flatten(item):
        yield x
    else:        
      yield item

In [33]:
def Triangle(Tournament):
  
  n,m = Tournament.shape
  
  for u in range(n):
    x = np.where(Tournament[u] == 1)
    for v in x[0]:
      y = np.where(Tournament[v] == 1)
      for w in y[0]:
        if Tournament[w][u]:
          return [u,v,w]
  
  return []

def Topologicalsort(Tournament, Vertices):
  
  n,_ = Tournament.shape
  degree = np.zeros(shape = n, dtype = int)
  
  for u in range(n):
    x= np.where(Tournament[u] == 1)
    for v in x[0]:
      degree[v]+=1
      
  queue=[]
  x= np.where(degree == 0)
  for u in x[0]:
    queue.append(u)
    
  count = 0
  Topologicalorder =[]
  
  while queue:
    u = queue.pop(0)
    Topologicalorder.append(Vertices[u])
    
    x= np.where(Tournament[u] == 1)
    for v in x[0]:
      degree[v]-=1
      if degree[v] == 0:
        queue.append(v)
    count+=1
      
  if count!=n:
    return (False, [])
  else:
    return (True, Topologicalorder)

In [35]:
def find(Tournament, Vertices, cycle):

  n, _ = Tournament.shape
  Splus, Sminus, A, B, C = [], [], [], [], []

  # cycle triangle with nodes as "a", "b", "c"
  for i in range(n):
    # if node beats every node in triangle add it to Splus
    if Tournament[i][cycle[0]] and Tournament[i][cycle[1]] and Tournament[i][cycle[2]]:
      Splus.append(Vertices[i])

    # if node is beaten by every node in triangele then add it to Sminus
    elif Tournament[cycle[0]][i] and Tournament[cycle[1]][i] and Tournament[cycle[2]][i]:
      Sminus.append(Vertices[i])

    # if node beats "b" and beaten by "c"
    elif Tournament[i][cycle[1]] and Tournament[cycle[2]][i]:
      A.append(Vertices[i])

    # if node beats "c" and beaten by "a"
    elif Tournament[i][cycle[2]] and Tournament[cycle[0]][i]:
      B.append(Vertices[i])

    # if node beats "a" and beaten by "b"
    elif Tournament[i][cycle[0]] and Tournament[cycle[1]][i]:
      C.append(Vertices[i])

  return Splus, Sminus, A, B, C

In [36]:
# counts total number of upsets in the predicted rank
def Upsets(Tournament, sigma):
  
  n, = sigma.shape
  count = 0

  # if the pairs (i, j) for which σ(j) < σ(i) but the fraction of
  # times i being preferred over j in the test set is ≥ 0.5
  for i in range(n):
    for j in range(i+1, n):
      if Tournament[i][j] != 1:
        count+=1
  return count

def MFAST(Tournament, Vertices):
  sigma = np.array(Vertices)
  n, = sigma.shape
  upset = n*n
  sigmaStar = sigma.copy()

  for i in range(n):
    rank_eval = Upsets(Tournament, sigma)
    if rank_eval<upset:
      upset = rank_eval
      sigmaStar = sigma
    sigma = np.roll(sigma, 1)
    Tournament = np.insert(Tournament, n, Tournament[0], axis=0)
    Tournament = np.delete(Tournament, 0, 0)
    Tournament = np.insert(Tournament, n, Tournament[:,0], axis=1)
    Tournament = np.delete(Tournament, 0, 1)

  return sigmaStar

In [ ]:
def Rank(Tournament, Vertices):
  return Vertices

In [37]:
def BR2R(Tournament, Vertices):

  Tour = Tournament.Sub_tournament(Vertices)
  n, _ = Tour.shape
  flag, Sort = Topologicalsort(Tour, Vertices)

  if flag:
    return Sort
  else:
    cycle = Triangle(Tour)
    if len(cycle)==0: return Rank(Tournament, Vertices)
    Splus, Sminus, A, B, C = find(Tour, Vertices, cycle)
    sigma = [BR2R(Tournament, A), BR2R(Tournament, B), BR2R(Tournament, C)]
    sigma1 = BR2R(Tournament, Splus)
    sigma2 = BR2R(Tournament, Sminus)
    sigma = list(flatten(sigma))
    sigma = MFAST(Tournament.Sub_tournament(sigma), sigma)

    return list(flatten(sigma1)) + list(sigma) + list(flatten(sigma2))

In [38]:
def PairwiseBlockrank(data, rank):
  players = Playerss(data)
  P = PPM(data, players)
  Mask = mask(P.copy())
  Matrix = Matrix_Completion(P, rank, Mask)
  Tour = Tournaments(Matrix)
  sp, _ = Tour.Tournament.shape
  Vertices = [i for i in range(sp)]
  rank = BR2R(Tour, Vertices)
  ups = Upsets(Tour.Sub_tournament(rank), np.array(rank))
  return [rank, ups]

In [40]:
result=[]
for i in range(20):
  temp=[]
  for r in [2,4,6,8]:
    temp.append(PairwiseBlockrank(data, r))
  result.append(temp)


In [ ]:
for i, item in enumerate(result):
  for g in range(4):
    print(f'In run {i+1} for rank {(g+1)*2} : Total upsets are {item[g][1]}')